# **`Celery`:**

## **`Distributed task Queues in General`:**

A **Distributed Task Queue (DTQ)** is a system that manages and coordinates tasks across multiple servers or machines. Instead of one machine handling all work, a DTQ spreads tasks to multiple workers, improving performance and reliability. **Key characteristics include:**


* **`Scalability` –** tasks can be processed in parallel by adding more worker nodes.

* **`Fault tolerance` –** if a worker fails, tasks can be retried or taken by another worker.

* **`Load balancing` –** tasks are distributed to prevent any single node from becoming a bottleneck.

* **`Asynchronous processing` –** tasks run independently of the main application flow, keeping the system responsive.

* **`Persistence and retries` –** tasks are stored in persistent queues or storages and retried according to policies.

* **`Monitoring and scheduling` –** tools monitor queue length, worker status and schedule recurring tasks.



### **`Typical Architecture`:**
A DTQ comprises the following components:

| **Component**                       | **Role**                                                                          | **Example technology**                      |
| ------------------------------- | ----------------------------------------------------------------------------- | --------------------------------------- |
| **Task producer**               | Generates tasks (e.g., web server, data pipeline). Sends them to the queue.   | Web application or microservice         |
| **Task queue / message broker** | Holds tasks until workers process them; ensures tasks are delivered reliably. | RabbitMQ, Kafka, Redis                  |
| **Workers (consumers)**         | Dequeue tasks and execute them; multiple workers enable parallelism.          | Celery workers, custom worker processes |
| **Result backend**              | Stores task outcomes so producers can query results (optional).               | Redis, database                         |
| **Scheduler**                   | Adds tasks to the queue at specific times for recurring jobs.                 | Celery beat, cron                       |
| **Monitoring & logging**        | Observes queue size, worker health and task success/failure.                  | Flower (Celery monitoring), Grafana     |



### **`Example of a Distributed task Queue`:**
Suppose a news website needs to process uploaded images (generate thumbnails) and send notifications. Instead of processing each upload synchronously, the application uses a distributed task queue (Celery). **The process looks like this:**

1. **Image upload –** When a user uploads an image via the website, the application immediately returns a response but sends two tasks to the queue: `generate_thumbnail(image_path)` and `send_notification(user_id)`.

2. **Message broker –** The tasks are placed in Redis. One queue may hold image‑processing tasks while another holds notification tasks.

3. **Workers –** Several worker nodes subscribed to the image queue generate thumbnails concurrently; another set of workers subscribed to the notification queue sends user notifications. Workers may run on different machines, enabling horizontal scaling.

4. **Result –** The generated thumbnails are stored in object storage (e.g., S3). The result (thumbnail URL) is stored in the result backend. The notification task returns a success status. The main application can poll results if needed.


This architecture provides resilience (failures on one worker do not affect others), scalability (more workers can be added for busy queues) and decoupling between task production and execution

## **`Key Points`:**

* **`Broker`:** A broker is an intermediary component in a distributed system that **receives, stores, and routes messages** between **producers** (send tasks to the broker) and **consumers** (process the tasks). In a task queue system, producers send tasks to the broker instead of directly to workers. The broker **queues these tasks** reliably until workers are available to process them. It ensures decoupling, so producers and workers do not need to know about each other’s state or availability. Brokers also support **message acknowledgment, retry, and persistence**, improving fault tolerance. Common brokers include Redis, RabbitMQ, and Amazon SQS.


* **`Decoupling`:** **Decoupling** is a design principle where system components are made **independent of each other**, so changes in one component do not directly impact others. In distributed systems, it means the producer of work does not need to know how, when, or where the work is processed. Components interact through well-defined interfaces (such as queues or APIs) instead of direct calls. This improves **scalability, reliability, and maintainability**. Decoupling also allows components to evolve, scale, or fail independently without breaking the entire system.


* **`Worker`:** A **worker** is a process or service that **executes tasks** taken from a queue, such as running background jobs or long-running computations. It continuously listens for new tasks, processes them, and optionally reports results or status.


* **`Worker Nodes`:** **Worker nodes** are the machines (physical or virtual) that host one or more workers. Multiple worker nodes allow tasks to be **processed in parallel**, improving throughput and scalability. If one worker or node fails, others can continue processing tasks, increasing system reliability.

## **`Celery Background Tasks`:**

### **`Background Tasks in Web Applications`:**
Modern web applications often perform work that is **resource‑intensive or time‑consuming** (e.g., sending email notifications, processing images, generating reports or making external API calls). Executing these tasks synchronously can block the main request–response cycle and slow down the user experience. A common solution is to move such work to **background jobs** that run asynchronously, leaving the main application thread free to respond quickly.



### **`Celery Overview`:**
**Celery** is a popular Python library that implements a **distributed task queue**. It allows developers to define background tasks as Python functions and run them asynchronously across one or more worker processes. Real‑time processing and scheduled jobs are both supported. **Key points about Celery include:**

* A task is created from any callable using the `@app.task` decorator. When called, the decorated function sends a **message** describing the task to a broker instead of executing immediately.

* **Workers** are processes that listen to the broker and execute pending tasks. They can be scaled across multiple machines to increase parallelism.

* A **message broker** (e.g., Redis or RabbitMQ) stores the queued tasks and delivers them to workers. Celery’s architecture works with different brokers, letting developers choose one that fits their environment.

* Celery uses a **result backend** to store task results if configured; results can be ignored to reduce overhead.

* `Celeryd` is the daemon that executes tasks; **Celery beat** is an optional scheduler that triggers periodic tasks at specified times.



### **`Why use Celery?`**
Running tasks asynchronously has two major benefits:
1. **Offloading work –** time‑consuming operations run in separate processes rather than blocking the main application.

2. **Scheduling tasks –** tasks can be triggered at specific times or recurring intervals.

Celery defines itself as “a task queue with focus on real‑time processing, while also supporting task scheduling”. Because workers run tasks independently of one another and outside the application process, Celery is widely used to keep web apps responsive while handling operations like sending emails, resizing images, performing analysis, or making third‑party API calls.


### **`How Celery Works`:**
#### **`Task Definition and Registration`:**
When a function is decorated with `@app.task`, Celery registers it in the application’s **task registry**.The registry maps task names to their classes. Tasks only become available when the module defining them is imported, so workers need the same code base as the client. <br>

Only the **task name** and arguments are sent to the broker—not the function’s code. Upon receiving a task message, the worker uses the name to look up the registered function and executes it. This decoupled message‑passing design enables distributed execution and ensures that tasks can survive worker failures. <br>


When we say **“Celery registers a task in the application’s task registry”**, it means this:
When you decorate a function with `@app.task`, Celery **records that function in an internal dictionary** called the task registry. This registry maps a **unique task** name (for example, `tasks.process_pdf`) to the actual Python function. <br>

Celery does **not store the code in Redis or RabbitMQ—**the code stays in your application.
Later, when a worker receives a task message, it uses the **task name** to look up the function in this registry and run it. <br><br>


* **`Question`:** I have python ocde in my application so when task is register that time function also register?
    * **Ans:** When you register a task with `@app.task`, **Celery registers the function’s reference** (its name and callable) in the task registry at import time. The function code itself stays in your Python application, and Celery just keeps a mapping so workers know **which function to execute when a task name arrives**.






## **`Step-by-Step Process How Celery Works`:**
Below is the simplest “anyone can understand” flow for **`PDF processing` + `email notification`** using Celery.


1. **You have 3 parts Running:**
    * Your **Web App** (FastAPI/Flask/Django).
    * A **Broker** (Redis/RabbitMQ) = the “waiting line”.
    * One or more **Workers** (Celery workers) = the “people who do the work”.

2. **You write two Tasks:**
    * **Task 1:** `process_pdf(file_id)`
    * **Task 2:** `send_email(user_id, message)`
    * You mark them with `@app.task`, so workers know these tasks exist.

3. **User uploads a PDF:**
    * Your web app receives the file and saves it (disk/S3/etc.)
    * Web app creates a `file_id` (like a receipt number)

4. **Web app submits the PDF job (does NOT process it):**
    * Web app calls: `process_pdf.delay(file_id)`
    * This tells Celery: “Run this later, in the background.”

5. **Celery sends a small message to the Broker:**
    * Message contains only: **task name** + **arguments** (example: `tasks.process_pdf`, `file_id`)
    * The actual code is not sent; only the “job ticket” is sent.

6. **Broker holds the job ticket in a Queue:**
    * The task waits there until a worker is free.

7. **A Worker picks the PDF job Ticket:**
    * Worker is always listening to the broker.
    * It takes the message from the queue.

8. **Worker runs the PDF Task:**
    * Worker reads the task name and finds the matching function in its task registry.
    * Then it executes: `process_pdf(file_id)`

9. **PDF task Finishes and Submits the Email Job:**
    * After processing, the task calls: `send_email.delay(user_id, "PDF processed")`
    * Again, Celery sends a new “job ticket” to the broker.

10. **A Worker picks the Email Job and Sends the Email:**
    * Worker runs `send_email(...)` and the notification is sent.

11. **Optional: track Status/Result**
    * If you use a result backend, Celery can store success/failure and return values.
    * If you don’t need results, you can disable storing them (`ignore_result=True`).



**In one line:** <br>
Your web app creates `“job tickets”` **→** `broker holds them` **→** `workers pick tickets and do the work` **→** `optionally store results`.

## **`Key Features of Celery`:**

1. **Distributed Task Queue:**
    * Celery distributes tasks across multiple machines using a broker (Redis/RabbitMQ).
    * **Example:** 1,000 PDF files are `uploaded` **→** `Celery` splits them into tasks processed by many workers in parallel.


2. **Asynchronous Tasks:**
    * Tasks run in the background without blocking the main application.
    * **Example:** `process_pdf.delay(file_id)` returns immediately while the PDF is processed later.


3. **Scheduled Tasks (Celery Beat):**
    * Celery can run tasks at fixed times or intervals.
    * **Example:** Send a daily report email every night at `11 PM` using Celery Beat.


4. **Distributed Workers:**
    * Multiple worker processes or nodes can execute tasks simultaneously.
    * **Example:** One worker processes PDFs, another sends emails, and another handles OCR—scaling independently as load increases.